### Improving the accuracy of the MNISTmodel.

In [1]:
import tensorflow as tf

In [2]:
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
# Parameters
learning_rate = 0.01
training_epochs = 10
batch_size = 100
display_step = 1
input_size = 784
output_size = 10

In [4]:
#keep probability
pkeep = tf.placeholder(tf.float32)
# tf Graph Input
x = tf.placeholder(tf.float32, [None, input_size]) # mnist data image of shape 28*28=784
y = tf.placeholder(tf.float32, [None, output_size]) # 0-9 digits recognition => 10 classes

In [6]:
dimensions = [200,100,60,30]
current_input = x
n_input = input_size

# We're going to keep every matrix we create so let's create a list to hold them all
Ws = []

# We'll create a for loop to create each layer:
for layer_i, n_output in enumerate(dimensions):

    # just like in the last session,
    # we'll use a variable scope to help encapsulate our variables
    # This will simply prefix all the variables made in this scope
    # with the name we give it.
    with tf.variable_scope("FC/layer/{}".format(layer_i),reuse=None):

        # Create a weight matrix which will increasingly reduce
        # down the amount of information in the input by performing
        # a matrix multiplication
        W = tf.get_variable(
            name='W',
            shape=[n_input, n_output],
            #initializer=tf.random_normal_initializer(mean=0.0, stddev=0.02)
            initializer = tf.contrib.layers.xavier_initializer(uniform=True, seed=None, dtype=tf.float32))
        
        B = tf.get_variable(
            name='B',
            shape=[n_output],
            #initializer=tf.random_normal_initializer(mean=0.0, stddev=0.02)
            initializer = tf.contrib.layers.xavier_initializer(uniform=True, seed=None, dtype=tf.float32))
        

        # Now we'll multiply our input by our newly created W matrix
        # and add the bias
        H = tf.nn.bias_add(
            name='H',
            value=tf.matmul(current_input, W),
            bias=B)

        # And then use a relu activation function on its output
        H  = tf.nn.relu(H)
        
        #add a dropout layer
        current_input = tf.nn.dropout(H,pkeep)

        # Finally we'll store the weight matrix so we can build the decoder.
        Ws.append(W)

        # We'll also replace n_input with the current n_output, so that on the
        # next iteration, our new number inputs will be correct.
        n_input = n_output
        
        print(current_input.get_shape().as_list())

[None, 200]
[None, 100]
[None, 60]
[None, 30]


In [8]:
## we have our final layer
with tf.variable_scope("FCFinal", reuse=None):
    Wfinal = tf.get_variable(
            name='Wfinal',
            shape=[dimensions[-1], output_size],
            #shape=[input_size, output_size],
            #initializer=tf.random_normal_initializer(mean=0.0, stddev=0.02)
            initializer = tf.contrib.layers.xavier_initializer(uniform=True, seed=None, dtype=tf.float32))

    Bfinal = tf.get_variable(
            name='Bfinal',
            shape=[output_size],
            #initializer=tf.random_normal_initializer(mean=0.0, stddev=0.02)
            initializer = tf.contrib.layers.xavier_initializer(uniform=True, seed=None, dtype=tf.float32))

    logits = tf.nn.bias_add(
            name='logits',
            value=tf.matmul(current_input, Wfinal),
            #value = tf.matmul(x, Wfinal),
            bias=Bfinal)
 
    pred = tf.nn.softmax(logits) # Softmax 

tf.summary.histogram("prediction", pred)
print(pred.get_shape().as_list())

[None, 10]


In [10]:
with tf.variable_scope("FCoptimize", reuse=None):
    
    #cross entropy error
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
    tf.summary.histogram("loss", cost)
    
    #adamoptimizer
    optimizer =  tf.train.AdamOptimizer(learning_rate).minimize(cost) 

In [11]:
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [13]:
# no of correct prediction
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
# Calculate accuracy
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.histogram("accuracy", accuracy)

<tf.Tensor 'accuracy:0' shape=() dtype=string>

In [15]:
sess = tf.Session()
# Run the initializer
sess.run(init)
#tensorboard writer
train_writer = tf.summary.FileWriter( './logs/1/train ', sess.graph)
counter=0
# Training cycle
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(mnist.train.num_examples/batch_size)
    # Loop over all batches
    for i in range(total_batch):
        counter+=1
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        #merge summary
        merge = tf.summary.merge_all()
        # Run optimization op (backprop) and cost op (to get loss value)
        summary, _, c = sess.run([merge, optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys, pkeep: 1.0})
        # Compute average loss
        avg_cost += c / total_batch
        train_writer.add_summary(summary, counter)
    # Display logs per epoch step
    if (epoch+1) % display_step == 0:
        print("Epoch:", '%04d' % (epoch+1), "cost= ",(avg_cost))
        acc = sess.run(accuracy,feed_dict = {x: batch_xs, y: batch_ys, pkeep: 1.0})
        print ("accuracy = ",acc)
        ##print("Accuracy:", acc.eval(sesion = sess))

print("Optimization Finished!")

Epoch: 0001 cost=  0.278455955434
accuracy =  0.96
Epoch: 0002 cost=  0.147857927458
accuracy =  0.97
Epoch: 0003 cost=  0.116440413835
accuracy =  0.99
Epoch: 0004 cost=  0.10783731399
accuracy =  1.0
Epoch: 0005 cost=  0.0955067597922
accuracy =  0.99
Epoch: 0006 cost=  0.0836281189297
accuracy =  1.0
Epoch: 0007 cost=  0.0858893141426
accuracy =  1.0
Epoch: 0008 cost=  0.077920345993
accuracy =  0.98
Epoch: 0009 cost=  0.0826905458378
accuracy =  0.98
Epoch: 0010 cost=  0.0744962864475
accuracy =  0.98
Optimization Finished!


In [16]:
##run it on test Set
acc1 = sess.run(accuracy,feed_dict = {x: mnist.test.images, 
                                      y: mnist.test.labels, pkeep: 1.0})
print ("accuracy on test set: ",acc1)

accuracy on test set:  0.9711


In [17]:
##this is my graph
g = tf.get_default_graph()
[op.name for op in g.get_operations()]

['Placeholder',
 'Placeholder_1',
 'Placeholder_2',
 'FC/layer/0/W/Initializer/random_uniform/shape',
 'FC/layer/0/W/Initializer/random_uniform/min',
 'FC/layer/0/W/Initializer/random_uniform/max',
 'FC/layer/0/W/Initializer/random_uniform/RandomUniform',
 'FC/layer/0/W/Initializer/random_uniform/sub',
 'FC/layer/0/W/Initializer/random_uniform/mul',
 'FC/layer/0/W/Initializer/random_uniform',
 'FC/layer/0/W',
 'FC/layer/0/W/Assign',
 'FC/layer/0/W/read',
 'FC/layer/0/B/Initializer/random_uniform/shape',
 'FC/layer/0/B/Initializer/random_uniform/min',
 'FC/layer/0/B/Initializer/random_uniform/max',
 'FC/layer/0/B/Initializer/random_uniform/RandomUniform',
 'FC/layer/0/B/Initializer/random_uniform/sub',
 'FC/layer/0/B/Initializer/random_uniform/mul',
 'FC/layer/0/B/Initializer/random_uniform',
 'FC/layer/0/B',
 'FC/layer/0/B/Assign',
 'FC/layer/0/B/read',
 'FC/layer/0_1/MatMul',
 'FC/layer/0_1/H',
 'FC/layer/0_1/Relu',
 'FC/layer/0_1/dropout/Shape',
 'FC/layer/0_1/dropout/random_uniform/